In [6]:
import pandas as pd
import torch

In [7]:
df = pd.read_csv('통영.csv', encoding='cp949')
df

,지점,일시,평균 풍속(m/s),평균기압(hPa),평균 상대습도(%),평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,22188,2015-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2,0.0,NaN,8.0
1,22188,2015-11-18,13.8,1017.7,85.0,15.4,17.5,4.9,3.0,4.9,7.2,7.4,9.1
2,22188,2015-11-19,8.7,1021.9,77.0,15.4,17.4,3.3,2.0,3.7,6.2,8.1,9.1
3,22188,2015-11-20,3.0,1022.5,77.0,15.7,17.3,1.4,0.8,1.2,2.3,6.5,7.1
4,22188,2015-11-21,6.1,1025.1,79.0,16.1,17.3,1.1,0.7,0.8,1.4,4.4,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,22188,2022-12-27,5.3,1026.7,53.0,6.6,13.7,1.3,0.9,1.0,1.7,4.2,4.7
2525,22188,2022-12-28,7.8,1028.5,60.0,5.7,13.2,1.5,0.9,1.3,2.1,3.7,3.9
2526,22188,2022-12-29,5.7,1031.3,56.0,4.8,13.2,1.2,0.8,1.1,1.8,3.6,4.1
2527,22188,2022-12-30,7.4,1031.2,57.0,5.5,13.4,1.2,0.8,1.0,1.6,3.5,3.7


In [8]:
df = df.rename(columns={'일시': 'date', '평균 기온(°C)': 'Atemp(°C)', '평균 수온(°C)': 'Stemp(°C)', '평균 최대 파고(m)':'wave(m)','평균 풍속(m/s)':'Wind(m/s)','평균기압(hPa)':'Apress(hPa)','평균 상대습도(%)':'Moisture(%)'})
df=df.set_index(pd.to_datetime(df.date)) # Date를 인덱스로
df=df[['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)']] # Column 재정리
df.dropna(subset=['Wind(m/s)','Apress(hPa)','Moisture(%)','Atemp(°C)','Stemp(°C)','wave(m)'], inplace=True)
df

,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
date,,,,,,
2015-11-18,13.8,1017.7,85.0,15.4,17.5,4.9
2015-11-19,8.7,1021.9,77.0,15.4,17.4,3.3
2015-11-20,3.0,1022.5,77.0,15.7,17.3,1.4
2015-11-21,6.1,1025.1,79.0,16.1,17.3,1.1
2015-11-22,8.1,1026.0,83.0,16.1,17.5,2.0
...,...,...,...,...,...,...
2022-12-27,5.3,1026.7,53.0,6.6,13.7,1.3
2022-12-28,7.8,1028.5,60.0,5.7,13.2,1.5
2022-12-29,5.7,1031.3,56.0,4.8,13.2,1.2


In [9]:
df = df.loc['2016-01-01':]
df

,Wind(m/s),Apress(hPa),Moisture(%),Atemp(°C),Stemp(°C),wave(m)
date,,,,,,
2016-01-01,5.4,1030.9,67.0,7.3,14.1,1.1
2016-01-02,2.6,1024.3,70.0,11.1,14.3,0.8
2016-01-03,4.2,1019.5,76.0,13.0,15.4,0.7
2016-01-04,6.2,1021.1,59.0,12.2,15.2,1.2
2016-01-05,4.4,1023.9,63.0,9.2,13.1,0.9
...,...,...,...,...,...,...
2022-12-27,5.3,1026.7,53.0,6.6,13.7,1.3
2022-12-28,7.8,1028.5,60.0,5.7,13.2,1.5
2022-12-29,5.7,1031.3,56.0,4.8,13.2,1.2


In [10]:
import pymysql
from sqlalchemy import create_engine, types

# Establish a connection to MySQL
host = 'localhost'
user = 'root'
password = 'k404'
database = 'forecast'

df = df.reset_index()

conn = pymysql.connect(host=host, user=user, password=password, database=database)

# Create a SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# Convert the DataFrame to a SQL table with specified column types
table_name = 'tongyoung'
dtype = {'date': types.Date(), 'Wind (m/s)': types.Float(), 'Apress (hPa)': types.Float(), 'Moisture (%)': types.Float(), 'Atemp (°C)': types.Float(), 'Stemp (°C)': types.Float(), 'wave (m)': types.Float()}
df.to_sql(table_name, engine, if_exists='replace', index=False, dtype=dtype)

# Close the connection
conn.close()
